<font size=5><b>Group Name: <font color=darkgreen>Suicide Squad</b></font></font>
<br>
<br>
<font size=5>**Members:**</font><br>
<br>
<font size=4>
**Ansel Santos**<br>
**Sal Lascano**<br>
**Moon Kang**<br>
**Yicong Xu**
</font>

# References for our work:
1. 

# Preparing the Packages to be used

Calling the packages we will use for the project, along with the plotly username and api key

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from scipy.stats import norm, skew
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
import seaborn as sns
import json
import requests
import warnings
from requests.auth import HTTPBasicAuth

np.random.seed(0)

def ignore_warn(*args, **kwargs):
    pass

warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

username = 'moonkang' # Replace with YOUR USERNAME
api_key = 'nKkigC26m95bqBAK52af' # Replace with YOUR API KEY

auth = HTTPBasicAuth(username, api_key)
headers = {'Plotly-Client-Platform': 'python'}

plotly.tools.set_credentials_file(username=username, api_key=api_key)

pd.set_option('display.max_columns', None)

This area is for deleting plots in plotly (run this block if the # of plots is already maxed out)

In [118]:
#for deleting plots in plotly
def get_pages(username, page_size):
    url = 'https://api.plot.ly/v2/folders/all?user='+username+'&page_size='+str(page_size)
    response = requests.get(url, auth=auth, headers=headers)
    if response.status_code != 200:
        return
    page = json.loads(response.content)
    yield page
    while True:
        resource = page['children']['next'] 
        if not resource:
            break
        response = requests.get(resource, auth=auth, headers=headers)
        if response.status_code != 200:
            break
        page = json.loads(response.content)
        yield page
        
def permanently_delete_files(username, page_size=500, filetype_to_delete='plot'):
    for page in get_pages(username, page_size):
        for x in range(0, len(page['children']['results'])):
            fid = page['children']['results'][x]['fid']
            res = requests.get('https://api.plot.ly/v2/files/' + fid, auth=auth, headers=headers)
            res.raise_for_status()
            if res.status_code == 200:
                json_res = json.loads(res.content)
                if json_res['filetype'] == filetype_to_delete:
                    # move to trash
                    requests.post('https://api.plot.ly/v2/files/'+fid+'/trash', auth=auth, headers=headers)
                    # permanently delete
                    requests.delete('https://api.plot.ly/v2/files/'+fid+'/permanent_delete', auth=auth, headers=headers)

permanently_delete_files(username, filetype_to_delete='plot')
permanently_delete_files(username, filetype_to_delete='grid')

# Loading the data

Load the data from .csv and looking at the head of the data and dimensions

In [3]:
## load training data
train = pd.read_csv("train.csv", header = 0, index_col=None)
## load test data
test = pd.read_csv("test.csv", header = 0, index_col=None)

print(train.shape)
print(test.shape)
train.head()

(1460, 81)
(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Exploratory Data Analysis

In [4]:
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Based on [Serigne's](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard) kernel, we are dropping the ID column.

In [5]:
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

Added the sum of Basement and Livable Area as a column for the scatterplot matrix

In [6]:
train['LivBsmtArea']=train['TotalBsmtSF'] + train['GrLivArea']
test['LivBsmtArea']=test['TotalBsmtSF'] + test['GrLivArea'] #also applied it to the test set

Building the Scatterplot Matrix, just for us to see the relationship of the target variable with variables we think will have a big impact to the price

In [7]:
dataframe = train[['SalePrice', 'GrLivArea', 'TotalBsmtSF', 'LivBsmtArea']].copy()

fig = ff.create_scatterplotmatrix(dataframe, height=1000, width=1000, diag='histogram', 
                                  size=3, title="House Train Variables")
py.iplot(fig)

Remove the outliers and log transform the sale price

In [8]:
#Outlier removal
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#Log transform
train["LogSalePrice"] = np.log1p(train["SalePrice"])

Plot the data without the outliers

In [11]:
dataframe = train[['LogSalePrice', 'LivBsmtArea']].copy()

fig = ff.create_scatterplotmatrix(dataframe, height=500, width=500, diag='histogram', 
                                  size=3, title="LogSalePrice vs LivBsmtArea")
py.iplot(fig)

In [12]:
SalePrice = go.Histogram(x=train['SalePrice'], cumulative=dict(enabled=True))
LogSalePrice = go.Histogram(x=train['LogSalePrice'], cumulative=dict(enabled=True))
fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(SalePrice, 1, 1)
fig.append_trace(LogSalePrice, 1, 2)

fig['layout'].update(height=350, width=750)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



# Correlation Matrix Heatmap

In [13]:
corrmat = train.corr()
y = list(corrmat.columns.values)
x = list(corrmat.index.values)
z = [list(corrmat[x]) for x in corrmat]

In [15]:
trace = go.Heatmap(z=z, x=x, y=y, colorscale='Viridis')
data=[trace]
fig['layout'].update(height=900, width=900)
py.iplot(data)

# Data Cleaning process

Adding a column which uses sales price

In [16]:
#add column which uses SalePrice/Log Sale Price
#changing MSSubClass
tmp1 = train.groupby(['MSSubClass'])['SalePrice'].mean()
tmp1 = np.log(tmp1)
tmp1 = dict(tmp1)
tmp1[150] = train.SalePrice.mean()
tmp1

{20: 12.12932556396575,
 30: 11.470328193887799,
 40: 11.958412247427878,
 45: 11.595349949344998,
 50: 11.872716354858824,
 60: 12.390074215675833,
 70: 12.024385387526202,
 75: 12.167526704666857,
 80: 12.042002818271586,
 85: 11.903682944206118,
 90: 11.802164401802564,
 120: 12.209960480395255,
 150: 180932.91906721535,
 160: 11.839689161232487,
 180: 11.535664951939717,
 190: 11.772310938266896}

Combining Train and Test Data sets. Ideally, we should apply the same process on a separate place to avoid data leakage, but for this project, we combined them and did the cleaning process to make our jupyter notebook shorter.

In [17]:
ntrain = train.shape[0]
ntest = test.shape[0]

saleprice_train = train.SalePrice.values
priceperft_train = train.SalePrice.values/train.LotArea.values
logsaleprice_train = train.LogSalePrice.values
lotarea_train = train.LotArea.values
lotarea_test = test.LotArea.values

all_df = pd.concat((train, test)).reset_index(drop=True)
all_df.drop(['SalePrice'], axis=1, inplace=True)
all_df.drop(['LogSalePrice'], axis=1, inplace=True)

print("Dimensions - {}".format(all_df.shape))

Dimensions - (2917, 80)


Counting the missingness on each column

In [18]:
all_df_na = (all_df.isnull().sum() / len(all_df)) * 100
all_df_na = all_df_na.drop(all_df_na[all_df_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_df_na})
missing_data.head(10)

data = [go.Bar(x=all_df_na.index, y=all_df_na, marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5
        )))]

layout = go.Layout(
    xaxis=dict(tickangle=-90), title="% of Missingness"
)

fig = go.Figure(data=data, layout=layout)
fig['layout'].update(height=400, width=800)

py.iplot(fig)

# Checking of lot frontage vs lot area

Why did we check this?

Among the variables which have the most missingness (PoolQC, MiscFeature, Alley, Fence, FireplaceQu and LotFrontage), Lot frontage was the one which had a high correlation with the sale price (Correlation vs SalePrice = 0.37).  That is why we had to study the variable further to make sure we are not doing the incorrect imputation.

In [20]:

trace = go.Scatter(
    x = all_df['LotFrontage'],
    y = all_df['LotArea'],
    mode = 'markers'
)

layout = go.Layout(
    xaxis=dict(
        title='Lot Frontage'),
    yaxis=dict(
        title='Lot Area')
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
fig['layout'].update(height=400, width=800)

py.iplot(fig)

In [21]:
index = all_df['LotFrontage'].index[all_df['LotFrontage'].apply(np.isnan)]
LFna_df = all_df.loc[index, :]
LFna_df.head()

#conclusion: the NaN in LotFrontage are zero because the column does not differentiate zero form NaN
#the column does not have zero as minimum based on describe
#NaN in lotfrontage = zero (fill with 0)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LivBsmtArea,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
7,1107,983,0,NaN,3,1Fam,TA,Mn,859.0,32.0,ALQ,BLQ,1.0,0.0,Gd,216.0,Y,PosN,Norm,SBrkr,228,TA,TA,HdBoard,HdBoard,NaN,TA,2,CBlock,2,Typ,484.0,2.0,TA,RFn,TA,Attchd,1973.0,2090,1,GasA,Ex,2Story,1,TA,Lvl,Gtl,3197.0,10382,Corner,NaN,IR1,0,60,RL,240.0,Stone,Shed,350,11,NWAmes,204,6,7,Y,0,NaN,CompShg,Gable,Normal,WD,0,Pave,7,1107.0,AllPub,235,1973,1973,2009
12,912,0,0,NaN,2,1Fam,TA,No,737.0,0.0,ALQ,Unf,1.0,0.0,TA,175.0,Y,Norm,Norm,SBrkr,0,TA,TA,HdBoard,Plywood,NaN,NaN,0,CBlock,1,Typ,352.0,1.0,TA,Unf,TA,Detchd,1962.0,912,0,GasA,TA,1Story,1,TA,Lvl,Gtl,1824.0,12968,Inside,NaN,IR2,0,20,RL,0.0,None,NaN,0,9,Sawyer,0,6,5,Y,0,NaN,CompShg,Hip,Normal,WD,176,Pave,4,912.0,AllPub,140,1962,1962,2008
14,1253,0,0,NaN,2,1Fam,TA,No,733.0,0.0,BLQ,Unf,1.0,0.0,TA,520.0,Y,Norm,Norm,SBrkr,176,TA,TA,MetalSd,MetalSd,GdWo,Fa,1,CBlock,1,Typ,352.0,1.0,TA,RFn,TA,Attchd,1960.0,1253,1,GasA,TA,1Story,1,TA,Lvl,Gtl,2506.0,10920,Corner,NaN,IR1,0,20,RL,212.0,BrkFace,NaN,0,5,NAmes,213,5,6,Y,0,NaN,CompShg,Hip,Normal,WD,0,Pave,5,1253.0,AllPub,0,1960,1960,2008
16,1004,0,0,NaN,2,1Fam,TA,No,578.0,0.0,ALQ,Unf,1.0,0.0,TA,426.0,Y,Norm,Norm,SBrkr,0,TA,TA,Wd Sdng,Wd Sdng,NaN,TA,1,CBlock,1,Typ,480.0,2.0,TA,Fin,TA,Attchd,1970.0,1004,0,GasA,Ex,1Story,1,TA,Lvl,Gtl,2008.0,11241,CulDSac,NaN,IR1,0,20,RL,180.0,BrkFace,Shed,700,3,NAmes,0,7,6,Y,0,NaN,CompShg,Gable,Normal,WD,0,Pave,5,1004.0,AllPub,0,1970,1970,2010
24,1060,0,0,NaN,3,1Fam,TA,Mn,188.0,668.0,Rec,ALQ,1.0,0.0,TA,204.0,Y,Norm,Norm,SBrkr,0,Gd,TA,Plywood,Plywood,MnPrv,TA,1,CBlock,1,Typ,270.0,1.0,TA,Unf,TA,Attchd,1968.0,1060,0,GasA,Ex,1Story,1,Gd,Lvl,Gtl,2120.0,8246,Inside,NaN,IR1,0,20,RL,0.0,None,NaN,0,5,Sawyer,90,8,5,Y,0,NaN,CompShg,Gable,Normal,WD,0,Pave,6,1060.0,AllPub,406,1968,2001,2010


# Filling the Missingness (Part 1)

In [22]:
#Handling features guided by data description
all_df.loc[:, "Alley"] = all_df.loc[:, "Alley"].fillna("None")
all_df.loc[:, "BsmtQual"] = all_df.loc[:, "BsmtQual"].fillna("NoBasmt")
all_df.loc[:, "BsmtCond"] = all_df.loc[:, "BsmtCond"].fillna("NoBasmt")
all_df.loc[:, "BsmtExposure"] = all_df.loc[:, "BsmtExposure"].fillna("NoBasmt")
all_df.loc[:, "BsmtFinType1"] = all_df.loc[:, "BsmtFinType1"].fillna("NoBasmt")
all_df.loc[:, "BsmtFinType2"] = all_df.loc[:, "BsmtFinType2"].fillna("NoBasmt")
all_df.loc[:, "BsmtFullBath"] = all_df.loc[:, "BsmtFullBath"].fillna(0)
all_df.loc[:, "BsmtHalfBath"] = all_df.loc[:, "BsmtHalfBath"].fillna(0)
all_df.loc[:, "BsmtUnfSF"] = all_df.loc[:, "BsmtUnfSF"].fillna(0)
all_df.loc[:, "Fence"] = all_df.loc[:, "Fence"].fillna("NoFnc")
all_df.loc[:, "FireplaceQu"] = all_df.loc[:, "FireplaceQu"].fillna("NoFrplc")
all_df.loc[:, "Fireplaces"] = all_df.loc[:, "Fireplaces"].fillna(0)
all_df.loc[:, "Functional"] = all_df.loc[:, "Functional"].fillna("Typ")
all_df.loc[:, "GarageType"] = all_df.loc[:, "GarageType"].fillna("NoGrg")
all_df.loc[:, "GarageFinish"] = all_df.loc[:, "GarageFinish"].fillna("NoGrg")
all_df.loc[:, "GarageQual"] = all_df.loc[:, "GarageQual"].fillna("NoGrg")
all_df.loc[:, "GarageCond"] = all_df.loc[:, "GarageCond"].fillna("NoGrg")
all_df.loc[:, "GarageArea"] = all_df.loc[:, "GarageArea"].fillna(0)
all_df.loc[:, "GarageCars"] = all_df.loc[:, "GarageCars"].fillna(0)
all_df.loc[:, "MiscFeature"] = all_df.loc[:, "MiscFeature"].fillna("NoMscFtr")
all_df.loc[:, "MiscVal"] = all_df.loc[:, "MiscVal"].fillna(0)
all_df.loc[:, "PoolQC"] = all_df.loc[:, "PoolQC"].fillna("NoPool")
all_df.loc[:, "PoolArea"] = all_df.loc[:, "PoolArea"].fillna(0)

#cleaning of other NaNs left behind (not done by our reference)
all_df.loc[:, "GarageYrBlt"] = all_df.loc[:, "GarageYrBlt"].fillna(all_df['YearBuilt'])
all_df.loc[:, "BsmtFinSF1"] = all_df.loc[:, "BsmtFinSF1"].fillna(0)
all_df.loc[:, "BsmtFinSF2"] = all_df.loc[:, "BsmtFinSF2"].fillna(0)
all_df.loc[:, "TotalBsmtSF"] = all_df.loc[:, "TotalBsmtSF"].fillna(0)
all_df.loc[:, "LivBsmtArea"] = all_df.loc[:, "LivBsmtArea"].fillna(all_df['TotalBsmtSF'] + all_df['GrLivArea'])


# Filling the Missingness (Part 2)

In [23]:
# By missing at random, decided to fill NA with 0 for BedroomAbvGr as missingness can be  
# interpreted to be directly related to lack of value
all_df.loc[:, "BedroomAbvGr"] = all_df.loc[:, "BedroomAbvGr"].fillna(0)

# By missing at random, decided to fill NA with N for CentralAir as missingness can be 
# interpreted to be directly related to lack of Central Air
all_df.loc[:, "CentralAir"] = all_df.loc[:, "CentralAir"].fillna("N")

# By missing at random, decided to fill NA with Norm for Condition1 & Condition2 as missingness 
# can be interpreted to be directly related to lack of proximity to any conditions
all_df.loc[:, "Condition1"] = all_df.loc[:, "Condition1"].fillna("Norm")
all_df.loc[:, "Condition2"] = all_df.loc[:, "Condition2"].fillna("Norm")

# By missing at random, decided to fill NA with 0 for EnclosedPorch as missingness can be  
# interpreted to be directly related to lack of value
all_df.loc[:, "EnclosedPorch"] = all_df.loc[:, "EnclosedPorch"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for ExterCond & ExterQual
# since we assume it will have minumum impact
all_df.loc[:, "ExterCond"] = all_df.loc[:, "ExterCond"].fillna("TA")
all_df.loc[:, "ExterQual"] = all_df.loc[:, "ExterQual"].fillna("TA")

# By missing at random, decided to fill NA with 0 for HalfBath as missingness can be 
# interpreted to be directly related to lack of value
all_df.loc[:, "HalfBath"] = all_df.loc[:, "HalfBath"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for HeatingQC 
# since we assume it will have minumum impact
all_df.loc[:, "HeatingQC"] = all_df.loc[:, "HeatingQC"].fillna("TA")

# By missing at random, decided to fill NA with 0 for KitchenAbvGr as missingness can be  
# interpreted to be directly related to lack of value
all_df.loc[:, "KitchenAbvGr"] = all_df.loc[:, "KitchenAbvGr"].fillna(0)

# By missing completely at random, decided to fill NA with TA (averaging) for KitchenQual 
# since we assume it will have minumum impact
all_df.loc[:, "KitchenQual"] = all_df.loc[:, "KitchenQual"].fillna("TA")

# *****LotFrontage : NA most likely means no lot frontage - missing at random*****
all_df.loc[:, "LotFrontage"] = all_df.loc[:, "LotFrontage"].fillna(0)

# By missing completely at random, decided to fill NA with Reg (averaging) for LotShape 
# since we assume it will have minumum impact
all_df.loc[:, "LotShape"] = all_df.loc[:, "LotShape"].fillna("Reg")

# By missing at random, decided to fill NA with None and 0 for MasVnrType & MasVnrArea 
# as missingness can be interpreted to be directly 
# related to lack of vaue
all_df.loc[:, "MasVnrType"] = all_df.loc[:, "MasVnrType"].fillna("None")
all_df.loc[:, "MasVnrArea"] = all_df.loc[:, "MasVnrArea"].fillna(0)

# By missing at random, decided to fill NA with 0 for OpenPorchSF as missingness can be  
# interpreted to be directly related to lack of vaue
all_df.loc[:, "OpenPorchSF"] = all_df.loc[:, "OpenPorchSF"].fillna(0)

# By missing at random, decided to fill NA with N for PavedDrive as missingness can be  
# interpreted to be directly related to No Paved Drive
all_df.loc[:, "PavedDrive"] = all_df.loc[:, "PavedDrive"].fillna("N")

# By missing completely at random, decided to fill NA with Normal (averaging) for SaleCondition 
# since we assume it will have minumum impact
all_df.loc[:, "SaleCondition"] = all_df.loc[:, "SaleCondition"].fillna("Normal")

# By missing at random, decided to fill NA with 0 for ScreenPorch as missingness can be  
# interpreted to be directly related to lack of value
all_df.loc[:, "ScreenPorch"] = all_df.loc[:, "ScreenPorch"].fillna(0)

# By missing at random, decided to fill NA with 0 for TotRmsAbvGrd as missingness can be  
# interpreted to be directly related to lack of value
all_df.loc[:, "TotRmsAbvGrd"] = all_df.loc[:, "TotRmsAbvGrd"].fillna(0)

# By missing at random, decided to fill NA with AllPub for Utilities as missingness can be 
# interpreted to be directly related to All public Utilities 
all_df.loc[:, "Utilities"] = all_df.loc[:, "Utilities"].fillna("AllPub")

# By missing at random, decided to fill NA with 0 for WoodDeckSF as missingness can be  
# interpreted to be directly related to lack of value
all_df.loc[:, "WoodDeckSF"] = all_df.loc[:, "WoodDeckSF"].fillna(0)


# Transformations (Part 1)

In [24]:
# There are variables with numerical vaues that after investigating in the Description we 
# find that are actually best described as categories
# Changing the month sold and Year sold variables into categorical is a great example of 
# using dummy variables to adjust for seasonality

# adding the MS SubClass2
all_df['MSSubClass2'] = [tmp1.get(key) for key in all_df.MSSubClass.values]

all_df = all_df.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

all_df['YrSold'] = ["Year" + str(x) for x in all_df['YrSold']]


# Transformations (Part 2)

In [25]:
# Encoding categorical features as ordered numbers after gathering 
# inside of order from description

all_df = all_df.replace({"Alley" : {"None": 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"NoBasmt" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"NoBasmt" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"NoBasmt" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"NoBasmt" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"NoBasmt" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"NoFrplc" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"NoGrg" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageQual" : {"NoGrg" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"NoPool" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}}
                     )

# Adding Simplified version of existing columns

In [26]:
# Create new features
# Simplifications of existing features
all_df["SimplOverallQual"] = all_df.OverallQual.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })
all_df["SimplOverallCond"] = all_df.OverallCond.replace({1 : 1, 2 : 1, 3 : 1, # bad
                                                       4 : 2, 5 : 2, 6 : 2, # average
                                                       7 : 3, 8 : 3, 9 : 3, 10 : 3 # good
                                                      })
all_df["SimplPoolQC"] = all_df.PoolQC.replace({1 : 1, 2 : 1, # average
                                             3 : 2, 4 : 2 # good
                                            })
all_df["SimplGarageCond"] = all_df.GarageCond.replace({1 : 1, # bad
                                                     2 : 1, 3 : 1, # average
                                                     4 : 2, 5 : 2 # good
                                                    })
all_df["SimplGarageQual"] = all_df.GarageQual.replace({1 : 1, # bad
                                                     2 : 1, 3 : 1, # average
                                                     4 : 2, 5 : 2 # good
                                                    })
all_df["SimplFireplaceQu"] = all_df.FireplaceQu.replace({1 : 1, # bad
                                                       2 : 1, 3 : 1, # average
                                                       4 : 2, 5 : 2 # good
                                                      })
all_df["SimplFireplaceQu"] = all_df.FireplaceQu.replace({1 : 1, # bad
                                                       2 : 1, 3 : 1, # average
                                                       4 : 2, 5 : 2 # good
                                                      })
all_df["SimplFunctional"] = all_df.Functional.replace({1 : 1, 2 : 1, # bad
                                                     3 : 2, 4 : 2, # major
                                                     5 : 3, 6 : 3, 7 : 3, # minor
                                                     8 : 4 # typical
                                                    })
all_df["SimplKitchenQual"] = all_df.KitchenQual.replace({1 : 1, # bad
                                                       2 : 1, 3 : 1, # average
                                                       4 : 2, 5 : 2 # good
                                                      })
all_df["SimplHeatingQC"] = all_df.HeatingQC.replace({1 : 1, # bad
                                                   2 : 1, 3 : 1, # average
                                                   4 : 2, 5 : 2 # good
                                                  })
all_df["SimplBsmtFinType1"] = all_df.BsmtFinType1.replace({1 : 1, # unfinished
                                                         2 : 1, 3 : 1, # rec room
                                                         4 : 2, 5 : 2, 6 : 2 # living quarters
                                                        })
all_df["SimplBsmtFinType2"] = all_df.BsmtFinType2.replace({1 : 1, # unfinished
                                                         2 : 1, 3 : 1, # rec room
                                                         4 : 2, 5 : 2, 6 : 2 # living quarters
                                                        })
all_df["SimplBsmtCond"] = all_df.BsmtCond.replace({1 : 1, # bad
                                                 2 : 1, 3 : 1, # average
                                                 4 : 2, 5 : 2 # good
                                                })
all_df["SimplBsmtQual"] = all_df.BsmtQual.replace({1 : 1, # bad
                                                 2 : 1, 3 : 1, # average
                                                 4 : 2, 5 : 2 # good
                                                })
all_df["SimplExterCond"] = all_df.ExterCond.replace({1 : 1, # bad
                                                   2 : 1, 3 : 1, # average
                                                   4 : 2, 5 : 2 # good
                                                  })
all_df["SimplExterQual"] = all_df.ExterQual.replace({1 : 1, # bad
                                                   2 : 1, 3 : 1, # average
                                                   4 : 2, 5 : 2 # good
                                                  })


# Combination of variables (additional Columns)

In [27]:
# 2* Combinations of existing features
# Overall quality of the house
all_df["OverallGrade"] = all_df["OverallQual"] * all_df["OverallCond"]
# Overall quality of the garage
all_df["GarageGrade"] = all_df["GarageQual"] * all_df["GarageCond"]
# Overall quality of the exterior
all_df["ExterGrade"] = all_df["ExterQual"] * all_df["ExterCond"]
# Overall kitchen score
all_df["KitchenScore"] = all_df["KitchenAbvGr"] * all_df["KitchenQual"]
# Overall fireplace score
all_df["FireplaceScore"] = all_df["Fireplaces"] * all_df["FireplaceQu"]
# Overall garage score
all_df["GarageScore"] = all_df["GarageArea"] * all_df["GarageQual"]
# Overall pool score
all_df["PoolScore"] = all_df["PoolArea"] * all_df["PoolQC"]
# Simplified overall quality of the house
all_df["SimplOverallGrade"] = all_df["SimplOverallQual"] * all_df["SimplOverallCond"]
# Simplified overall quality of the exterior
all_df["SimplExterGrade"] = all_df["SimplExterQual"] * all_df["SimplExterCond"]
# Simplified overall pool score
all_df["SimplPoolScore"] = all_df["PoolArea"] * all_df["SimplPoolQC"]
# Simplified overall garage score
all_df["SimplGarageScore"] = all_df["GarageArea"] * all_df["SimplGarageQual"]
# Simplified overall fireplace score
all_df["SimplFireplaceScore"] = all_df["Fireplaces"] * all_df["SimplFireplaceQu"]
# Simplified overall kitchen score
all_df["SimplKitchenScore"] = all_df["KitchenAbvGr"] * all_df["SimplKitchenQual"]
# Total number of bathrooms
all_df["TotalBath"] = all_df["BsmtFullBath"] + (0.5 * all_df["BsmtHalfBath"]) + \
all_df["FullBath"] + (0.5 * all_df["HalfBath"])

# Total SF for 1st + 2nd floors
all_df["AllFlrsSF"] = all_df["1stFlrSF"] + all_df["2ndFlrSF"]
# Total SF for porch
all_df["AllPorchSF"] = all_df["OpenPorchSF"] + all_df["EnclosedPorch"] + \
all_df["3SsnPorch"] + all_df["ScreenPorch"]
# Has masonry veneer or not
all_df["HasMasVnr"] = all_df.MasVnrType.replace({"BrkCmn" : 1, "BrkFace" : 1, "CBlock" : 1, 
                                               "Stone" : 1, "None" : 0})
# House completed before sale or not
all_df["BoughtOffPlan"] = all_df.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
                                                      "Family" : 0, "Normal" : 0, "Partial" : 1})


# Plots checking the relationship with SalesPrice

In [42]:
plot_ss = train[['SalePrice']]


trace1 = go.Box(x=list(plot1['SimplOverallQual']), y=list(plot_ss['SalePrice']), 
                jitter=0.75, marker=dict(size=4))
trace2 = go.Box(x=list(plot1['OverallQual']), y=list(plot_ss['SalePrice']), 
                jitter=0.75, marker=dict(size=4))
trace3 = go.Box(x=list(plot1['SimplOverallCond']), y=list(plot_ss['SalePrice']), 
                jitter=0.75, marker=dict(size=4))
trace4 = go.Box(x=list(plot1['OverallCond']), y=list(plot_ss['SalePrice']), 
                jitter=0.75, marker=dict(size=4))

fig = tools.make_subplots(rows=2, cols=2, subplot_titles=('Plot 1', 'Plot 2',
                                                          'Plot 3', 'Plot 4')) 

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)

fig['layout'].update(showlegend=False, height=800, width=800, title='Multiple Subplots with Titles')

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [100]:
plot_data = all_df[['SimplOverallQual','OverallQual',
                'SimplOverallCond', 'OverallCond',
                'SimplPoolQC', 'PoolQC',
                'SimplGarageCond','GarageCond',
                'SimplGarageQual', 'GarageQual',
                'SimplFireplaceQu', 'FireplaceQu',
                'SimplFunctional', 'Functional',
                'SimplKitchenQual', 'KitchenQual',
                'SimplHeatingQC', 'HeatingQC',
                'SimplBsmtFinType1', 'BsmtFinType1',
                'SimplBsmtFinType2', 'BsmtFinType2',
                'SimplBsmtCond', 'BsmtCond',
                'SimplBsmtQual', 'BsmtQual',
                'SimplExterCond', 'ExterCond',
                'SimplExterQual', 'ExterQual',
                'MSSubClass2', 'MSSubClass']][:ntrain]

In [108]:
x_data = [x for x in plot_data['MSSubClass'].unique()]
x_data

['SC60',
 'SC20',
 'SC70',
 'SC50',
 'SC190',
 'SC45',
 'SC90',
 'SC120',
 'SC30',
 'SC85',
 'SC80',
 'SC160',
 'SC75',
 'SC180',
 'SC40']

In [119]:
def boxplot_col(col_name, height, width):
    x_data = [x for x in plot_data[col_name].unique()]
    x_data.sort()
    y_data = [[x for x,y in zip(logsaleprice_train, plot_data[col_name]) if y==i] for i in x_data]

    traces = []

    for xd, yd in zip(x_data, y_data):
            traces.append(go.Box(
                y=yd,
                name=xd,
                boxpoints='all',
                marker=dict(
                    size=1,
                ),
                line=dict(width=1),
            ))

    fig = go.Figure(data=traces)
    fig['layout'].update(showlegend=False, height=height, width=width, title=col_name + " vs. LogSalePrice")
    return fig

In [114]:
py.iplot(boxplot_col('SimplOverallQual', 400, 400))

In [115]:
py.iplot(boxplot_col('OverallQual', 400, 800))

In [121]:
py.iplot(boxplot_col('MSSubClass', 400, 1000))

# Box Cox Transformation for skewed features

In [223]:
numeric_feats = all_df.dtypes[all_df.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_df[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

,Skew
MSSubClass2,53.981481
PoolScore,22.357445
SimplPoolScore,22.332942
MiscVal,21.939672
PoolQC,20.341424
SimplPoolQC,18.985317
PoolArea,17.688664
LotArea,13.109495
LowQualFinSF,12.084539
3SsnPorch,11.372080


In [224]:
skewness = skewness[abs(skewness) > 0.5]
print("{} features to Box Cox Transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    all_df[feat] = boxcox1p(all_df[feat], lam)

88 features to Box Cox Transform


# Label Encoding - this is commented out, will not need

In [129]:
#all_df.head(10)

In [130]:
# from sklearn.preprocessing import LabelEncoder
# cols = ('BldgType', 'BsmtExposure', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'Exterior1st',
#        'Exterior2nd', 'Fence', 'Foundation', 'GarageFinish', 'GarageType', 'Heating', 'HouseStyle',
#        'LandContour', 'LotConfig', 'MSSubClass', 'MSZoning', 'MasVnrType', 'MiscFeature', 'MoSold',
#        'Neighborhood', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'YrSold')
# # process columns, apply LabelEncoder to categorical features
# for c in cols:
#     lbl = LabelEncoder() 
#     lbl.fit(list(all_df[c].values)) 
#     all_df[c] = lbl.transform(list(all_df[c].values))

# # shape        
# print('Shape all_data: {}'.format(all_df.shape))

# Dummifying

In [225]:
all_df = pd.get_dummies(all_df, drop_first=False)
print(all_df.shape)

(2917, 293)


# Setting Columns to Use

In [16]:
# keep_cols = ['1stFlrSF', '3SsnPorch', 'BsmtFinSF1', 'BsmtFinType1', 'BsmtQual', 
#              'ExterQual', 'FullBath', 'Functional', 'GarageCars', 'GrLivArea', 'HalfBath', 'HeatingQC',
#              'KitchenQual', 'LivBsmtArea', 'LotArea', 'MasVnrArea', 'OpenPorchSF', 'OverallQual',
#              'PavedDrive', 'ScreenPorch', 'WoodDeckSF', 'SimplExterQual', 'OverallGrade', 'FireplaceScore',
#              'GarageScore', 'PoolScore', 'SimplOverallGrade', 'SimplGarageScore', 'TotalBath', 'BldgType_1Fam',
#              'BldgType_TwnhsE', 'BsmtExposure_3', 'Condition1_Norm', 'Condition1_PosN', 'Exterior1st_BrkFace',
#              'Exterior1st_MetalSd', 'Exterior2nd_CmentBd', 'Exterior2nd_VinylSd', 'Fence_NoFnc', 'Foundation_PConc',
#              'GarageFinish_Fin', 'GarageFinish_NoGrg', 'GarageType_BuiltIn', 'GarageType_NoGrg', 'LandContour_HLS',
#              'LotConfig_Corner', 'LotConfig_CulDSac', 'MSSubClass_SC20', 'MSSubClass_SC60', 'MSZoning_FV',
#              'MasVnrType_Stone', 'MoSold_Jul', 'MoSold_Jun', 'MoSold_May', 'Neighborhood_BrkSide',
#              'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_NoRidge', 'Neighborhood_NridgHt',
#              'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'RoofMatl_WdShngl', 'RoofStyle_Hip',
#              'SaleCondition_Normal', 'SaleType_New']
# all_df = all_df[keep_cols]

# Dropping Columns

In [226]:
drop_cols = ['MSSubClass2']

all_df = all_df.drop(drop_cols, axis=1)

# Re-assigning to Test and Train

In [227]:
print (ntrain)

1458


In [228]:
train = all_df[:ntrain]
test = all_df[ntrain:]

train.to_csv('train_cleaned.csv')
test.to_csv('test_cleaned.csv')

In [229]:
print (train.shape)
train.head()

(1458, 292)


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,EnclosedPorch,ExterCond,ExterQual,FireplaceQu,Fireplaces,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageQual,GarageYrBlt,GrLivArea,HalfBath,HeatingQC,KitchenAbvGr,KitchenQual,LandSlope,LivBsmtArea,LotArea,LotFrontage,LotShape,LowQualFinSF,MasVnrArea,MiscVal,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SimplOverallQual,SimplOverallCond,SimplPoolQC,SimplGarageCond,SimplGarageQual,SimplFireplaceQu,SimplFunctional,SimplKitchenQual,SimplHeatingQC,SimplBsmtFinType1,SimplBsmtFinType2,SimplBsmtCond,SimplBsmtQual,SimplExterCond,SimplExterQual,OverallGrade,GarageGrade,ExterGrade,KitchenScore,FireplaceScore,GarageScore,PoolScore,SimplOverallGrade,SimplExterGrade,SimplPoolScore,SimplGarageScore,SimplFireplaceScore,SimplKitchenScore,TotalBath,AllFlrsSF,AllPorchSF,HasMasVnr,BoughtOffPlan,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtExposure_0,BsmtExposure_1,BsmtExposure_2,BsmtExposure_3,BsmtExposure_No,CentralAir_N,CentralAir_Y,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NoFnc,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,GarageFinish_Fin,GarageFinish_NoGrg,GarageFinish_RFn,GarageFinish_Unf,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGrg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,MSSubClass_SC120,MSSubClass_SC150,MSSubClass_SC160,MSSubClass_SC180,MSSubClass_SC190,MSSubClass_SC20,MSSubClass_SC30,MSSubClass_SC40,MSSubClass_SC45,MSSubClass_SC50,MSSubClass_SC60,MSSubClass_SC70,MSSubClass_SC75,MSSubClass_SC80,MSSubClass_SC85,MSSubClass_SC90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MiscFeature_Gar2,MiscFeature_NoMscFtr,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_Apr,MoSold_Aug,MoSold_Dec,MoSold_Feb,MoSold_Jan,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Some

In [230]:
print (test.shape)
test.head()

(1459, 292)


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,EnclosedPorch,ExterCond,ExterQual,FireplaceQu,Fireplaces,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageQual,GarageYrBlt,GrLivArea,HalfBath,HeatingQC,KitchenAbvGr,KitchenQual,LandSlope,LivBsmtArea,LotArea,LotFrontage,LotShape,LowQualFinSF,MasVnrArea,MiscVal,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SimplOverallQual,SimplOverallCond,SimplPoolQC,SimplGarageCond,SimplGarageQual,SimplFireplaceQu,SimplFunctional,SimplKitchenQual,SimplHeatingQC,SimplBsmtFinType1,SimplBsmtFinType2,SimplBsmtCond,SimplBsmtQual,SimplExterCond,SimplExterQual,OverallGrade,GarageGrade,ExterGrade,KitchenScore,FireplaceScore,GarageScore,PoolScore,SimplOverallGrade,SimplExterGrade,SimplPoolScore,SimplGarageScore,SimplFireplaceScore,SimplKitchenScore,TotalBath,AllFlrsSF,AllPorchSF,HasMasVnr,BoughtOffPlan,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtExposure_0,BsmtExposure_1,BsmtExposure_2,BsmtExposure_3,BsmtExposure_No,CentralAir_N,CentralAir_Y,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NoFnc,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,GarageFinish_Fin,GarageFinish_NoGrg,GarageFinish_RFn,GarageFinish_Unf,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGrg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,MSSubClass_SC120,MSSubClass_SC150,MSSubClass_SC160,MSSubClass_SC180,MSSubClass_SC190,MSSubClass_SC20,MSSubClass_SC30,MSSubClass_SC40,MSSubClass_SC45,MSSubClass_SC50,MSSubClass_SC60,MSSubClass_SC70,MSSubClass_SC75,MSSubClass_SC80,MSSubClass_SC85,MSSubClass_SC90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MiscFeature_Gar2,MiscFeature_NoMscFtr,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_Apr,MoSold_Aug,MoSold_Dec,MoSold_Feb,MoSold_Jan,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Some

# Modeling

In [137]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import ensemble
import xgboost as xgb
import lightgbm as lgb

In [138]:
index = all_df.index[np.sum(np.isnan(all_df), axis=1)>0]
all_df_na = all_df.loc[index, :]
all_df_na.head(10)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,EnclosedPorch,ExterCond,ExterQual,Fireplaces,FullBath,Functional,GarageArea,GarageCars,GarageYrBlt,GrLivArea,HalfBath,KitchenAbvGr,KitchenQual,LandSlope,LivBsmtArea,LotArea,LotFrontage,LotShape,LowQualFinSF,MasVnrArea,MiscVal,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,SimplOverallQual,SimplOverallCond,SimplPoolQC,SimplGarageCond,SimplGarageQual,SimplFireplaceQu,SimplFunctional,SimplHeatingQC,SimplBsmtFinType1,SimplBsmtFinType2,SimplBsmtCond,SimplExterCond,SimplExterQual,OverallGrade,GarageScore,SimplOverallGrade,SimplExterGrade,SimplPoolScore,SimplGarageScore,SimplFireplaceScore,SimplKitchenScore,TotalBath,AllFlrsSF,AllPorchSF,HasMasVnr,BoughtOffPlan,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtExposure_0,BsmtExposure_1,BsmtExposure_2,BsmtExposure_3,BsmtExposure_No,CentralAir_N,CentralAir_Y,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NoFnc,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,GarageFinish_Fin,GarageFinish_NoGrg,GarageFinish_RFn,GarageFinish_Unf,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NoGrg,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,MSSubClass_SC120,MSSubClass_SC150,MSSubClass_SC160,MSSubClass_SC180,MSSubClass_SC190,MSSubClass_SC20,MSSubClass_SC30,MSSubClass_SC40,MSSubClass_SC45,MSSubClass_SC50,MSSubClass_SC60,MSSubClass_SC70,MSSubClass_SC75,MSSubClass_SC80,MSSubClass_SC85,MSSubClass_SC90,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MiscFeature_Gar2,MiscFeature_NoMscFtr,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_Apr,MoSold_Aug,MoSold_Dec,MoSold_Feb,MoSold_Jan,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofS

## Cross Validation Function

In [231]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, logsaleprice_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

def rmsle_cv_sqft(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, priceperft_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

## Lasso Regression

In [140]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
lasso_sqft = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

## Elastic Net

In [141]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
ENet_sqft = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

## Ridge Regression 

In [142]:
#could not get this to work
#has infinity problem
RidgeReg = make_pipeline(RobustScaler(), linear_model.Ridge(alpha = 1000))

## Gradient Boosting Regression

In [143]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

## XGBoost

In [144]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1)
                             #random_state =7, nthread = -1)

## LightGBM 

In [145]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

## Random Forest

In [146]:
model_rf = ensemble.RandomForestRegressor(n_estimators=50)

# Model Scores

In [147]:
# for filtering columns which are important based on lasso
lasso2 = Lasso(alpha =0.0005, random_state=1)
lasso2.fit(train, logsaleprice_train)
colnames = list(train)
significant_col = lasso2.coef_==0
colnames = [x for x,y in zip(colnames, significant_col) if y==True]
print (len(colnames))
print (colnames)

166
['Alley', 'BsmtFullBath', 'BsmtHalfBath', 'ExterCond', 'Fireplaces', 'GarageYrBlt', 'LandSlope', 'LotShape', 'OverallCond', 'PoolArea', 'Street', 'TotRmsAbvGrd', 'Utilities', 'YearBuilt', 'YearRemodAdd', 'SimplOverallQual', 'SimplOverallCond', 'SimplPoolQC', 'SimplGarageCond', 'SimplGarageQual', 'SimplFireplaceQu', 'SimplFunctional', 'SimplBsmtFinType2', 'SimplBsmtCond', 'SimplExterGrade', 'AllFlrsSF', 'HasMasVnr', 'BoughtOffPlan', 'BldgType_2fmCon', 'BldgType_Twnhs', 'BsmtExposure_0', 'BsmtExposure_2', 'Condition1_Feedr', 'Condition1_PosA', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'Condition2_Artery', 'Condition2_Feedr', 'Condition2_Norm', 'Condition2_PosA', 'Condition2_PosN', 'Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn', 'Electrical_FuseA', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_Mix', 'Exterior1st_AsbShng', 'Exterior1st_AsphShn', 'Exterior1st_BrkComm', 'Exterior1st_CBlock', 'Exterior1st_CemntBd', 'Exterior1st_HdBoard', 'Exterior1st_ImStucc'

## Log Sale Price as target

In [232]:
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 0.1116 (0.0045)

ElasticNet score: 0.1117 (0.0046)



In [233]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

Xgboost score: 0.1124 (0.0072)

LGBM score: 0.1127 (0.0063)



In [234]:
score = rmsle_cv(RidgeReg)
print("\nRidge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Ridge score: 19.1156 (30.5115)



In [235]:
score = rmsle_cv(GBoost)
print("\nGradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Gradient Boosting score: 0.1148 (0.0062)



In [236]:
score = rmsle_cv(model_rf)
print("\nRandom Forest score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))


Random Forest score: 0.1353 (0.0048)



## Sale Price per Square Foot as target

In [169]:
#for sqft measurements
score = rmsle_cv_sqft(lasso_sqft)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv_sqft(ENet_sqft)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


Lasso score: 4.1933 (0.3919)

ElasticNet score: 4.1777 (0.3864)



# Stacking

In [238]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=6):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [239]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet, lasso, GBoost, model_rf),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("\nStacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


Stacking Averaged models score: 0.1066 (0.0054)


In [172]:
stacked_averaged_models_sqft = StackingAveragedModels(base_models = (ENet_sqft, lasso_sqft),
                                                 meta_model = lasso_sqft)

score = rmsle_cv_sqft(stacked_averaged_models_sqft)
print("\nStacking Averaged models Sqft score: {:.4f} ({:.4f})".format(score.mean(), score.std()))


Stacking Averaged models Sqft score: 4.1401 (0.3933)


# Applying the models to test set

In [240]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [241]:
stacked_averaged_models.fit(train.values, logsaleprice_train)
stacked_train_pred = stacked_averaged_models.predict(train.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))
stacked_pred = stacked_pred.round(2)
print(rmsle(logsaleprice_train, stacked_train_pred))

0.0749091932882


In [208]:
stacked_averaged_models_sqft.fit(train.values, priceperft_train)
stacked_train_pred_sqft = stacked_averaged_models_sqft.predict(train.values)
stacked_pred_sqft = stacked_averaged_models_sqft.predict(test.values)
stacked_pred_sqft = stacked_pred_sqft.round(6)
print(rmsle(priceperft_train, stacked_train_pred_sqft))

3.28018135717


In [209]:
submission_sqft = stacked_pred_sqft*lotarea_test

In [242]:
Submission = pd.DataFrame({'Id':list(range(1461,2920,1)),'SalePrice':stacked_pred})
Submission.to_csv('submission_mar_2_2018_18h19m.csv', index=False)

In [180]:
Submission = pd.DataFrame({'Id':list(range(1461,2920,1)),'SalePrice':submission_sqft})
Submission.to_csv('submission_sqft_mar_2_2018_17h37m.csv', index=False)

In [244]:
stacked_pred

array([ 119027.68,  157764.92,  187003.45, ...,  161048.36,  117531.53,
        218538.26])

# The End

Thank you everyone for listening to Suicide Squad's presentation! :)